In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import torch
import pandas
import numpy as np

In [2]:
#import pandas
#dataset taken from https://www.kaggle.com/yashsawarn/wifi-stretgth-for-rooms

def read_dataset(csv_name = 'sao-paulo-properties-april-2019.csv'):
    """
    Reads a csv dataset 
    returns it as a pytorch tensor
    """
    data_frame = pandas.read_csv(csv_name)

    """
    Creates a new column with categories using the "price" column
      1) 900000 < y , categoría 4
      2) 580000 < y < 900000, categoría 3
      3) 400000 < y < 580000, categoría 2
      4) y < 400000, categoría 1    
    """
    data_frame.loc[ data_frame['Price'] <  400000, 'Class'] = 1
    data_frame.loc[(data_frame['Price'] >= 400000) & (data_frame['Price'] < 580000), 'Class'] = 2
    data_frame.loc[(data_frame['Price'] >= 580000) & (data_frame['Price'] < 900000), 'Class'] = 3
    data_frame.loc[ data_frame['Price'] >= 900000, 'Class'] = 4
    data_frame['Class'] = data_frame['Class'].astype('int')
    
    return data_frame


In [5]:
class Node_CART:    
    def __init__(self, num_classes = 4, ref_CART = None, current_depth = 0):
        """
        Create the node attributes
        param num_classes: K number of classes to classify
        param ref_cart: reference to the tree containing the node
        param current_depth: current depth of the node in the tree
        """
        self.ref_CART = ref_CART
        self.threshold_value = 0
        self.feature_num = 0
        self.node_right = None
        self.node_left = None
        self.data_torch_partition = None
        self.gini = 0
        self.dominant_class = None #Atributo con Mayor Cantidad de apariciones en el dataset
        self.accuracy_dominant_class = None        
        self.num_classes = num_classes
        self.current_depth = current_depth
    
    def to_xml(self, current_str = ""):
        """
        Recursive function to write the node content to an xml formatted string
        param current_str : the xml content so far in the whole tree
        return the string with the node content
        """
        str_node = "<node><thresh>" + str(self.threshold_value) + "</thresh>" + "<feature>" + str(self.feature_num) + "</feature><depth>" + str(self.current_depth)+ "</depth>" 
        str_node += "<gini>" + str(self.gini) + "</gini>"
        if(self.node_right != None):
            str_left = self.node_right.to_xml(current_str)
            str_node += str_left
        if(self.node_left != None):
            str_right = self.node_left.to_xml(current_str)
            str_node += str_right
            
        if(self.is_leaf()):
            str_node += "<dominant_class>" + str(self.dominant_class) + "</dominant_class><acc_dominant_class>"  + str(self.accuracy_dominant_class) + "</acc_dominant_class>"
        str_node += "</node>"
        return str_node
    
    def is_leaf(self):
        """
        Checks whether the node is a leaf
        """
        return (self.node_left == None and self.node_right == None)
    
    def create_with_children(self, data_torch, current_depth, list_selected_features = [], min_gini = 0.000001):
        """
        Creates a node by selecting the best feature and threshold, and if needed, creating its children
        param data_torch: dataset with the current partition to deal with in the node
        param current_depth: depth counter for the node
        param list_selected_features: list of selected features so far for the CART building process
        param min_gini: hyperparmeter selected by the user defining the minimum tolerated gini coefficient for a  node
        return the list of selected features so far
        """        
        #update depth of children
        depth_children = current_depth + 1
        if(depth_children <= self.ref_CART.get_max_depth()):
            num_observations = data_torch.shape[0]            
            #careful with max depth
            #if no threshold and feature were selected, select it using a greedy approach            
            (threshold_value, feature_num, gini) = self.select_best_feature_and_thresh(data_torch, list_features_selected = list_selected_features)
            list_selected_features += [feature_num]
            #store important data in attributes
            self.threshold_value = threshold_value
            self.feature_num = feature_num
            self.data_torch_partition = data_torch
            self.gini = gini            
            num_features = data_torch.shape[1]
            #data_torch_left = torch.zeros(1, num_features)
            #data_torch_right = torch.zeros(1, num_features)
            #create the right and left node data if the current gini is still high            
            if(self.gini > min_gini):                
                data_torch_left = data_torch[data_torch[:, feature_num] < threshold_value]
                data_torch_right = data_torch[data_torch[:, feature_num] >= threshold_value]
                #if the new partitions have more than min_observations, make them
                if(data_torch_left.shape[0] >= self.ref_CART.get_min_observations() and data_torch_right.shape[0] >= self.ref_CART.get_min_observations()):
                    #add data to the right and left children
                    self.node_right = Node_CART(num_classes = self.num_classes, ref_CART = self.ref_CART, current_depth = depth_children)
                    self.node_left = Node_CART(num_classes = self.num_classes, ref_CART = self.ref_CART, current_depth = depth_children)
                    list_selected_features = self.node_right.create_with_children(data_torch_right, depth_children, list_selected_features = list_selected_features)            
                    self.node_left.create_with_children( data_torch_left, depth_children, list_selected_features = list_selected_features)
        #if is leaf, fill the         
        if(self.is_leaf()):            
            labels_data = data_torch[:,  -1]
            self.dominant_class = torch.mode(labels_data).values.item()
            num_obs_label = labels_data[labels_data == self.dominant_class].shape[0]
            self.accuracy_dominant_class = num_obs_label / labels_data.shape[0]           
            
        return list_selected_features
    
 #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   
    def select_best_feature_and_thresh(self, data_torch, list_features_selected = [], num_classes = 4):
        """
        ONLY USE  2 FORS
        Selects the best feature and threshold that minimizes the gini coefficient
        param data_torch: dataset partition to analyze
        param list_features_selected list of features selected so far, thus must be ignored 
        param num_classes: number of K classes to discriminate from 
        return min_thresh, min_feature, min_gini found for the dataset partition when 
        selecting the found feature and threshold
        """       

        
        #TODO
        #Defining null vector for Ginis
        giniVec=pandas.DataFrame(columns=[data_torch.columns])

        #display(giniVec)
        #print("nodo_test: Min Threshold = %s, Feature=%s, Min Gini=%s"%(nodo_test.select_best_feature_and_thresh(dataset_cut)))

        for col in data_torch.columns:
            nClasses = data_torch.groupby([data_torch[[col]].columns.item()]).size().array
            giniVec[col] = [self.calculate_gini(data_torch[[col]], num_classes=nClasses)]
            print("\n Valores unicos de %s, %s:"%(col, data_torch[col].sort_values().unique()))
            print('Cantidad por valores \n', data_torch.groupby([col]).size())
            
            #Defining null vector for Thresholds per column
            threshVec=pandas.DataFrame(columns=data_torch[col].sort_values().unique())
            #print("Vector nulo para threshold de la columna: ", col)
            #print("Vector threshold : ", threshVec)
            
            #Calculates the threshold
            for val in data_torch[col].sort_values().unique() :
                filter_ = data_torch[col]<val
                
                valGiniI = self.calculate_gini(data_torch[col])
                #valGiniD =
                #threshVec[val]=
                

            #threshVec[col] = [len(nClasses)] Esto es un error
            
        #display(giniVec)
        """
        print('Min Gini value : ', giniVec[min(giniVec)].item())
        print('Min feature value : ', min(giniVec)[0])
        print('Min Threshold value : ', threshVec[min(giniVec)].item())
        display(threshVec)
        """
        return (len(nClasses), min(giniVec)[0], giniVec[min(giniVec)].item())   
        
    
    def calculate_gini(self, data_partition_torch, num_classes = 4):
        """
        Calculates the gini coefficient for a given partition with the given number of classes
        param data_partition_torch: current dataset partition as a tensor
        param num_classes: K number of classes to discriminate from
        returns the calculated gini coefficient
        """
        #TODO
        gini = 1 - sum((num_classes / data_partition_torch.shape[0]) **2)
        #print("calculate_gini => num_classes: ", num_classes, data_partition_torch.shape[0])
        #print("Result Gini: ", gini)

        return gini
    
   
    
    def evaluate_node(self, input_torch): 
        """
        Evaluates an input observation within the node. 
        If is not a leaf node, send it to the corresponding node
        return predicted label
        """
        feature_val_input = input_torch[self.feature_num]
        if(self.is_leaf()):
            return self.dominant_class
        else:
            if(feature_val_input < self.threshold_value):
                return self.node_left.evaluate_node(input_torch)
            else:
                return self.node_right.evaluate_node(input_torch)
        

#*******************************************************************************************************************

class CART:
   
    def __init__(self, dataset_torch, max_CART_depth = 4, min_observations = 2):
        """
        CART has only one root node
        """
        #min observations per node
        self.min_observations = min_observations
        self.root = Node_CART(num_classes = 4, ref_CART = self, current_depth = 0)
        self.max_CART_depth = max_CART_depth
        self.list_selected_features = []
        
    def get_root(self):
        """
        Gets tree root
        """
        return self.root
    
    def get_min_observations(self):
        """
        return min observations per node
        """
        return self.min_observations
    
    def get_max_depth(self):
        """
        Gets the selected max depth of the tree
        """
        return self.max_CART_depth
    
    def build_CART(self, data_torch):
        """
        Build CART from root
        """
        self.list_selected_features = self.root.create_with_children(data_torch, current_depth = 0)
    
    def to_xml(self, xml_file_name):
        """
        write Xml file with tree content
        """
        str_nodes = self.root.to_xml()
        print('str_nodes ', str_nodes)
        file = open(xml_file_name,"w+") 
        file.write(str_nodes)
        file.close()
        return str_nodes
    
    
    def evaluate_input(self, input_torch):
        """
        Evaluate a specific input in the tree and get the predicted class
        """
        return self.root.evaluate_node(input_torch)
        
    
def train_CART(dataset_torch, name_xml = "", max_CART_depth = 3, min_obs_per_leaf = 2): 
    """
    Train CART model
    """
    tree = CART(dataset_torch = dataset_torch, max_CART_depth = max_CART_depth, min_observations =  min_obs_per_leaf)
    tree.build_CART(dataset_torch)
    if(not name_xml == ""):
        tree.to_xml(name_xml)
    return tree

def test_CART(tree, testset_torch):
    """
    Test a previously built CART
    """
    #TODO, use tree.evaluate_input(current_observation) for this
    return accuracy



In [7]:
"""
Main function
"""
def main():

    dataset_torch = read_dataset()
    """
    display(dataset_torch)
    print(dataset_torch.describe())
    print("size full dataset: ",dataset_torch.size)
    """
    
    dataset_cut=dataset_torch[['Rooms','Size','Toilets','Parking']].copy()
    

    """
    display(dataset_cut)
    print(dataset_cut.describe())
    """
    #print('dataset_cut size: ',dataset_cut.size)
    
    giniBest = pandas.DataFrame(columns=['col', 'thresh','giniW'])
    display(giniBest)
    i=0
    for col in dataset_cut.columns :
        giniWVal=pandas.DataFrame([])
        for val in dataset_cut[col].sort_values().unique() :
            totCol = dataset_cut[col].shape[0]
            filter_ = dataset_cut[col]<val
            Xi = pandas.DataFrame(dataset_cut[col][filter_].copy())
            Xd = pandas.DataFrame(dataset_cut[col][~filter_].copy())
            #print('\n Col=%s, Val=%s, Xi :\n %s'%(col, val, Xi))
            #print('\n Col=%s, Val=%s, Xd :\n %s'%(col, val, Xd))
        
            #Calculates for each value in the column the left and right gini and the weighted gini per value
            if Xi.empty :
                giniI = 0.0
                            #self.calculate_gini(data_torch[col]) #Modificar para usar esta funcion
                qtyClassD = Xd.groupby([Xd[[col]].columns.item()]).size().array
                totQtyD   = Xd.shape[0]
                giniD = 1 - sum((qtyClassD/totQtyD)**2)
                giniW = giniD
            elif Xd.empty :
                giniD = 0.0
                qtyClassI = Xi.groupby([Xi[[col]].columns.item()]).size().array
                totQtyI   = Xi.shape[0]
                giniI = 1 - sum((qtyClassI/totQtyI)**2)
                giniW = giniI
            else :
                qtyClassI = Xi.groupby([Xi[[col]].columns.item()]).size().array
                totQtyI   = Xi.shape[0]
                giniI = 1 - sum((qtyClassI/totQtyI)**2)
                qtyClassD = Xd.groupby([Xd[[col]].columns.item()]).size().array
                totQtyD   = Xd.shape[0]
                giniD = 1 - sum((qtyClassD/totQtyD)**2)
                giniW = ((totQtyI/totCol)*giniI)+((totQtyD/totCol)*giniD)
            #Gets the minimun value for weighted gini
            if giniWVal.empty :
                giniWVal['Thresh']= [val]
                giniWVal['GiniW'] = [giniW]
            else :
                if giniW < giniWVal['GiniW'].item() :
                    giniWVal['Thresh']= [val]
                    giniWVal['GiniW'] = [giniW]
            
                
            print('\n Col=%s, Val=%s, XiShape=%s, XdShape=%s, giniI=%s, giniD=%s, ginigW=%s'%(col, val, Xi.shape[0], Xd.shape[0], giniI, giniD, giniW))
            print('Valores en giniWVal: \n', giniWVal)
        inter = pandas.DataFrame([[col, giniWVal['Thresh'].item(), giniWVal['GiniW'].item()]], columns=['col', 'thresh','giniW'])    
        giniBest=pandas.concat([giniBest, inter], ignore_index=True)
        display(giniBest)
        
        print('Index : ', giniBest.index[giniBest['giniW'] == min(giniBest.giniW)].item())
        
    #tree = train_CART(dataset_torch, name_xml = "CART_example.xml")
    #acc = test_CART(tree, dataset_torch)
    
    """
    CART_1 =  CART(dataset_torch)
    CART_1.to_xml("arbolito_vacio.xml")
    nodo_A = Node_CART(num_classes = 2, current_depth = 1)
    CART_1.root.node_left = nodo_A
    CART_1.to_xml("arbolito_peque.xml")

    #print(CART_1.ref_CART)
    print(CART_1.threshold_value)
    print(CART_1.feature_num)
    print(CART_1.node_right)
    print(CART_1.node_left)
    print(CART_1.data_torch_partition)
    print(CART_1.gini)
    print(CART_1.dominant_class)
    print(CART_1.accuracy_dominant_class)
    print(CART_1.num_classes)
    print(CART_1.current_depth)
    #nClasses = data_torch.groupby([data_torch[[col]].columns.item()]).size().array
    X = dataset_torch.groupby([dataset_torch[['Rooms']].columns.item()]).size().array
    Q = dataset_torch.shape[0]
    print('Valores unicos de Rooms:', dataset_torch['Rooms'].sort_values().unique())
    print('Cantidad por valores de Rooms', dataset_torch.groupby(['Rooms']).size())
    print('Valores unicos de Toilets:', dataset_torch['Toilets'].sort_values().unique())
    print('Cantidad por Valores de Toilets', dataset_torch.groupby(['Toilets']).size())
    print('Valores unicos de Size:', dataset_torch['Size'].sort_values().unique())
    print('Cantidad por Valores de Size', dataset_torch.groupby(['Size']).size())
    print('Valores unicos de Parking:', dataset_torch['Parking'].sort_values().unique())
    print('Cantidad de Valores de Parking', dataset_torch.groupby(['Parking']).size())
    print("Vector X \n ", X, "\n Total suma \n", X.sum(), "\n Cantidad Rows \n", dataset_torch.shape[0])
    print("Gini \n", 1- sum((X/Q)**2))
    gini = 1 - sum((X/Q)**2)
    """
    
main()    

,col,thresh,giniW



 Col=Rooms, Val=1, XiShape=0, XdShape=4894, giniI=0.0, giniD=0.6298064951637569, ginigW=0.6298064951637569
Valores en giniWVal: 
    Thresh     GiniW
0       1  0.629806

 Col=Rooms, Val=2, XiShape=436, XdShape=4458, giniI=0.0, giniD=0.5634195917793932, ginigW=0.5132252840524182
Valores en giniWVal: 
    Thresh     GiniW
0       2  0.513225

 Col=Rooms, Val=3, XiShape=2635, XdShape=2259, giniI=0.2761725278958984, giniD=0.24733802971185437, ginigW=0.26286293831728064
Valores en giniWVal: 
    Thresh     GiniW
0       3  0.262863

 Col=Rooms, Val=4, XiShape=4569, XdShape=325, giniI=0.5800846254179668, giniD=0.04815147928994079, ginigW=0.5447600907854354
Valores en giniWVal: 
    Thresh     GiniW
0       3  0.262863

 Col=Rooms, Val=5, XiShape=4886, XdShape=8, giniI=0.6285953384317577, giniD=0.21875, ginigW=0.6279253828315424
Valores en giniWVal: 
    Thresh     GiniW
0       3  0.262863

 Col=Rooms, Val=6, XiShape=4893, XdShape=1, giniI=0.6296552059150639, giniD=0.0, ginigW=0.6295265473

,col,thresh,giniW
0,Rooms,3,0.262863


Index :  0

 Col=Size, Val=30, XiShape=0, XdShape=4894, giniI=0.0, giniD=0.9868289887002074, ginigW=0.9868289887002074
Valores en giniWVal: 
    Thresh     GiniW
0      30  0.986829

 Col=Size, Val=31, XiShape=12, XdShape=4882, giniI=0.0, giniD=0.986770202002826, ginigW=0.9843506592108289
Valores en giniWVal: 
    Thresh     GiniW
0      31  0.984351

 Col=Size, Val=32, XiShape=22, XdShape=4872, giniI=0.49586776859504145, giniD=0.9867200496924889, ginigW=0.9845135212527374
Valores en giniWVal: 
    Thresh     GiniW
0      31  0.984351

 Col=Size, Val=33, XiShape=26, XdShape=4868, giniI=0.6153846153846153, giniD=0.9866988918289178, ginigW=0.9847262373157278
Valores en giniWVal: 
    Thresh     GiniW
0      31  0.984351

 Col=Size, Val=34, XiShape=34, XdShape=4860, giniI=0.7197231833910034, giniD=0.9866577757455672, ginigW=0.9848033057537293
Valores en giniWVal: 
    Thresh     GiniW
0      31  0.984351

 Col=Size, Val=35, XiShape=49, XdShape=4845, giniI=0.7713452728029988, giniD=0.98658


 Col=Size, Val=80, XiShape=3014, XdShape=1880, giniI=0.9700391757289253, giniD=0.9877512449071979, ginigW=0.9768431581676569
Valores en giniWVal: 
    Thresh     GiniW
0      51  0.970575

 Col=Size, Val=81, XiShape=3076, XdShape=1818, giniI=0.970828520988026, giniD=0.9880645930379617, ginigW=0.9772312956073115
Valores en giniWVal: 
    Thresh     GiniW
0      51  0.970575

 Col=Size, Val=82, XiShape=3103, XdShape=1791, giniI=0.9712582575530899, giniD=0.9879292859857324, ginigW=0.9773591590493838
Valores en giniWVal: 
    Thresh     GiniW
0      51  0.970575

 Col=Size, Val=83, XiShape=3142, XdShape=1752, giniI=0.971813272145803, giniD=0.9878814297033006, ginigW=0.9775655018231091
Valores en giniWVal: 
    Thresh     GiniW
0      51  0.970575

 Col=Size, Val=84, XiShape=3190, XdShape=1704, giniI=0.9724287300635803, giniD=0.9879825706980537, ginigW=0.9778442887969565
Valores en giniWVal: 
    Thresh     GiniW
0      51  0.970575

 Col=Size, Val=85, XiShape=3212, XdShape=1682, giniI=0.9

...

Index :  0

 Col=Toilets, Val=1, XiShape=0, XdShape=4894, giniI=0.0, giniD=0.5732317112987404, ginigW=0.5732317112987404
Valores en giniWVal: 
    Thresh     GiniW
0       1  0.573232

 Col=Toilets, Val=2, XiShape=905, XdShape=3989, giniI=0.0, giniD=0.4090917860667349, ginigW=0.33344240592975183
Valores en giniWVal: 
    Thresh     GiniW
0       2  0.333442

 Col=Toilets, Val=3, XiShape=3916, XdShape=978, giniI=0.3553889858635013, giniD=0.6482094839014557, ginigW=0.4139052194313638
Valores en giniWVal: 
    Thresh     GiniW
0       2  0.333442

 Col=Toilets, Val=4, XiShape=4368, XdShape=526, giniI=0.47118686601859683, giniD=0.5222643091558357, ginigW=0.4766765952973438
Valores en giniWVal: 
    Thresh     GiniW
0       2  0.333442

 Col=Toilets, Val=5, XiShape=4687, XdShape=207, giniI=0.536087701143424, giniD=0.290135125673878, ginigW=0.525684721347307
Valores en giniWVal: 
    Thresh     GiniW
0       2  0.333442

 Col=Toilets, Val=6, XiShape=4859, XdShape=35, giniI=0.5670967168364804

,col,thresh,giniW
0,Rooms,3,0.262863
1,Size,51,0.970575
2,Toilets,2,0.333442


Index :  0

 Col=Parking, Val=0, XiShape=0, XdShape=4894, giniI=0.0, giniD=0.5480646593770775, ginigW=0.5480646593770775
Valores en giniWVal: 
    Thresh     GiniW
0       0  0.548065

 Col=Parking, Val=1, XiShape=150, XdShape=4744, giniI=0.0, giniD=0.5200332042747172, ginigW=0.5040943034489699
Valores en giniWVal: 
    Thresh     GiniW
0       1  0.504094

 Col=Parking, Val=2, XiShape=3127, XdShape=1767, giniI=0.09133649187896786, giniD=0.37885922795743765, ginigW=0.19514782711612683
Valores en giniWVal: 
    Thresh     GiniW
0       2  0.195148

 Col=Parking, Val=3, XiShape=4492, XdShape=402, giniI=0.4673297695953388, giniD=0.5287492883839509, ginigW=0.4723748547103821
Valores en giniWVal: 
    Thresh     GiniW
0       2  0.195148

 Col=Parking, Val=4, XiShape=4731, XdShape=163, giniI=0.5172370332155753, giniD=0.2835635515073959, ginigW=0.5094542834161406
Valores en giniWVal: 
    Thresh     GiniW
0       2  0.195148

 Col=Parking, Val=5, XiShape=4867, XdShape=27, giniI=0.54305922371

,col,thresh,giniW
0,Rooms,3,0.262863
1,Size,51,0.970575
2,Toilets,2,0.333442
3,Parking,2,0.195148


Index :  3
